In [1]:
# Team DSTI - ISIC 2024

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Import all necessary packages
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import h5py
from PIL import Image
import io
import torchvision.models as models

# Disable version checking for albumentations
A.core.transforms_interface.is_module_available = lambda _: False

# Load Test Data
# Define paths
test_hdf5_path = '/kaggle/input/isic-2024-challenge/test-image.hdf5'
test_metadata_path = '/kaggle/input/isic-2024-challenge/test-metadata.csv'

# Load metadata
df_test = pd.read_csv(test_metadata_path)

# Define the SqueezeNetWithVSURF model
class SqueezeNetWithVSURF(nn.Module):
    def __init__(self, num_classes=1, vsurf_size=5, dropout_rate=0.5):
        super(SqueezeNetWithVSURF, self).__init__()
        
        # Load pre-trained SqueezeNet
        self.squeezenet = models.squeezenet1_1(pretrained=False)
        
        # Replace the last convolutional layer
        self.squeezenet.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=1)
        
        # Additional layers for VSURF features
        self.vsurf_fc = nn.Sequential(
            nn.Linear(vsurf_size, 32),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        
        # Combine features
        self.final_fc = nn.Linear(num_classes + 32, num_classes)

    def forward(self, x, vsurf):
        # SqueezeNet forward pass
        x = self.squeezenet(x)
        x = x.view(x.size(0), -1)
        
        # VSURF features forward pass
        vsurf = self.vsurf_fc(vsurf)
        
        # Combine features
        combined = torch.cat((x, vsurf), dim=1)
        
        # Final classification
        out = self.final_fc(combined)
        
        return out

# Define Dataset Class and Transformation
class ISICDataset(Dataset):
    def __init__(self, hdf5_file, df, transform=None):
        self.hdf5_file = h5py.File(hdf5_file, 'r')
        self.df = df
        self.transform = transform
        self.vsurf_columns = ['clin_size_long_diam_mm', 'tbp_lv_H', 'tbp_lv_deltaLBnorm', 'tbp_lv_perimeterMM', 'tbp_lv_Hext']

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        isic_id = self.df.iloc[idx]['isic_id']
        img_bytes = self.hdf5_file[isic_id][()]
        img = Image.open(io.BytesIO(img_bytes))
        img = np.array(img)

        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']

        # Convert VSURF features to float, replacing any non-numeric values with 0
        vsurf_features = self.df.iloc[idx][self.vsurf_columns].values
        vsurf_features = np.array([float(val) if isinstance(val, (int, float)) else 0.0 for val in vsurf_features])
        vsurf_features = torch.tensor(vsurf_features, dtype=torch.float32)

        return img, vsurf_features

    def __del__(self):
        self.hdf5_file.close()

# Load Pretrained Model
def load_model(model_path):
    model = SqueezeNetWithVSURF(num_classes=1, vsurf_size=5, dropout_rate=0.5)
    state_dict = torch.load(model_path, map_location='cpu')  # Load to CPU to avoid CUDA issues
    model.load_state_dict(state_dict)
    model.eval()
    return model

# Prediction function
def predict(model, test_loader, device):
    model.eval()
    preds = []
    
    with torch.no_grad():
        for inputs, vsurf_features in test_loader:
            inputs = inputs.to(device)
            vsurf_features = vsurf_features.to(device)
            outputs = model(inputs, vsurf_features)
            preds.append(torch.sigmoid(outputs).cpu().numpy())
    
    return np.concatenate(preds).flatten()

# Make Prediction
import logging
logging.basicConfig(level=logging.INFO)

if __name__ == "__main__":
    try:
        # Set up paths
        test_hdf5_path = '/kaggle/input/isic-2024-challenge/test-image.hdf5'
        test_metadata_path = '/kaggle/input/isic-2024-challenge/test-metadata.csv'
        model_path = '/kaggle/input/isic-2024-squeezenet-bes/best_model.pth'  # Update this path

        # Set device
        device = torch.cuda.is_available() and 'cuda' or 'cpu'
        print(f"Using device: {device}")

        # Load test data
        df_test = pd.read_csv(test_metadata_path)
        print(f"Loaded test data with {len(df_test)} samples")

        # Define transforms
        def get_transforms(*, data):
            if data == 'valid':
                return A.Compose([
                    A.Resize(224, 224),
                    A.Normalize(
                        mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225],
                    ),
                    ToTensorV2(),
                ])

        # Create test dataset and dataloader
        test_dataset = ISICDataset(test_hdf5_path, df_test, transform=get_transforms(data='valid'))
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
        print("Created test dataloader")

        # Load model
        model = load_model(model_path)
        model = model.to(device)
        print("Loaded model")

        # Make predictions
        predictions = predict(model, test_loader, device)
        print(f"Made predictions for {len(predictions)} samples")

        # Create output DataFrame with raw probabilities
        output_df = pd.DataFrame({
            'isic_id': df_test['isic_id'],
            'target': predictions  # Use the raw prediction probabilities
        })

        # Save the predictions to a CSV file for submission
        output_df.to_csv('submission.csv', index=False)
        print("Submission file created!")
        
        # Display the first few rows
        print("First few rows of predictions:")
        print(output_df.head())

        print("Execution completed successfully")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

Using device: cuda
Loaded test data with 3 samples
Created test dataloader


/tmp/ipykernel_37/2581120580.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location='cpu')  # Load to CPU to avoid CUDA issue

Loaded model
Made predictions for 3 samples
Submission file created!
First few rows of predictions:
        isic_id    target
0  ISIC_0015657  0.000474
1  ISIC_0015729  0.011559
2  ISIC_0015740  0.000343
Execution completed successfully
